In [1]:
#Get data for streets, driving, biking, walk paths
import osmnx as ox

#Library for analyziing complex networks and graphs
import networkx as nx

#Converts address to latitude and longitude
from geopy.geocoders import Nominatim

#Interactive Graphs
import folium

#Static Graphing
import matplotlib.pyplot as plt

#Display final map
from IPython.display import display

#Methods to get Geopy.geocoders to work
import ssl
import certifi

In [2]:
# Create an SSL context that uses certifi's certificate bundle
ssl_context = ssl.create_default_context(cafile=certifi.where())

#Initialize Geolocater for the cordinates
geolocator = Nominatim(user_agent='wichita_nav', ssl_context=ssl_context)

# Generate road network for Wichita
G = ox.graph_from_place('Wichita, Kansas, USA', network_type='drive')

# Function to plot paths on the map
def plot_route(G, route, route_map, color='blue'):
    route_points = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
    folium.PolyLine(route_points, color=color, weight=4.5, opacity=0.7).add_to(route_map)

#Function to get nearest node from a location
def get_nearest_node(address): 
    location = geolocator.geocode(address, timeout=10)  # 5 sec timeout 
    if location:
        print(f"Geocoded Address: {address} -> ({location.latitude}, {location.longitude})")  # Debugging output
        return ox.distance.nearest_nodes(G, X=location.longitude, Y=location.latitude)
    else: 
        print(f"Failed to geocode address: {address}")  # Identify the issue
        return None

# Example start and end addresses
start_address = "7331 Ayesbury Cir, Wichita KS"
end_address = "123 Douglas Ave, Wichita, KS"

start_node = get_nearest_node(start_address)
end_node = get_nearest_node(end_address)

if start_node and end_node:
    #interactive map
    route_map = folium.Map(location=[37.6872, -97.3301], zoom_start=12)  # Centering on Wichita

    #Route 1 shortest path using Dijkstra's algorithm
    shortest_route = nx.shortest_path(G, start_node, end_node, weight='length')
    plot_route(G, shortest_route, route_map, color='blue')

    # **Route 2: Second shortest path (Yen’s Algorithm)**
    G_simple = nx.DiGraph(G)

    k_paths = list(nx.shortest_simple_paths(G_simple, start_node, end_node, weight='length'))
    if len(k_paths) > 1:
        second_route = k_paths[1]  # Second best route
        plot_route(G, second_route, route_map, color='red')
    else: 
        second_route = None
    
    # **Route 3: Alternative path by modifying edge weights**
    # G_alt = G.copy()
    # for u, v, d in G_alt.edges(data=True):
    #     d['length'] *= 1.2  # Slightly increase all distances to favor alternate paths

    # alt_route = nx.shortest_path(G_alt, start_node, end_node, weight='length')
    # plot_route(G, alt_route, route_map, color='green')

    #Add markers for start and end points
    folium.Marker([(G.nodes[start_node]['y'], G.nodes[start_node]['x'])], popup='Start').add_to(route_map)
    folium.Marker([(G.nodes[end_node]['y'], G.nodes[end_node]['x'])], popup='End').add_to(route_map)

     # Display map
    display(route_map)
else:
    print("Error: One of the addresses could not be geocoded.")


    

Geocoded Address: 7331 Ayesbury Cir, Wichita KS -> (37.727137306122444, -97.25127428571429)
Geocoded Address: 123 Douglas Ave, Wichita, KS -> (37.68591687755102, -97.33855526530613)


KeyboardInterrupt: 